In [17]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from pandas import Series, DataFrame, Panel
import matplotlib.pyplot as plt
from matplotlib import collections as mc 
import os

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

# Tunable Variables
days_to_process = 7
pd.options.display.max_rows = 1000

# Loading data
col_names = ['TrackID','FrameNo', 'X', 'Y']
day1 = pd.read_table("001_20130101.txt",delimiter=" ", header=None, names=col_names)
day2 = pd.read_table("001_20130102.txt",delimiter=" ", header=None, names=col_names)
day3 = pd.read_table("001_20130103.txt",delimiter=" ", header=None, names=col_names)
day4 = pd.read_table("001_20130104.txt",delimiter=" ", header=None, names=col_names)
day5 = pd.read_table("001_20130105.txt",delimiter=" ", header=None, names=col_names)
day6 = pd.read_table("001_20130106.txt",delimiter=" ", header=None, names=col_names)
day7 = pd.read_table("001_20130107.txt",delimiter=" ", header=None, names=col_names)

frames = [day1, day2, day3, day4, day5, day6, day7]
concatDay = pd.concat(frames, ignore_index=True)
# concatDay = concatDay.sort_values(by='TrackID', ascending=True)
concatDay



,TrackID,FrameNo,X,Y
0,157,1251,95,290
1,157,1252,99,284
2,157,1253,100,288
3,157,1254,98,290
4,157,1255,95,293
5,157,1256,90,296
6,157,1257,85,298
7,157,1258,81,301
8,157,1259,75,302
9,157,1260,69,305


In [21]:
# Prepare .tra extension file for TraClus clustering algorithm
outputFile = []
concatDay = concatDay[['TrackID', 'X', 'Y']]
concatDayWithoutID = concatDay[['X', 'Y']]
concatDayWithID = concatDay[['TrackID']]
concatDayWithID = concatDayWithID.drop_duplicates('TrackID')
concatDayWithID = concatDayWithID.values  #it's an array now


# slicing the dataframe based on ID
row_iterator = concatDay.iterrows()
row_id, this_row = next(row_iterator)
#slicing dataframe to respective arrays for LineCollection function
initialID = this_row['TrackID']
slicingArray = []

####################suspected bug################################
for i, rows in row_iterator:
    if rows['TrackID'] != initialID:
        slicingArray.append(i)
        initialID = rows['TrackID']
        
# print(slicingArray)
newList = np.split( concatDayWithoutID, slicingArray )

####################suspected bug################################

#Now: We need to combine every single piece of data to conform with .tra
#header: number of dimensions
#      : number of trajectories
number_of_dimension = 2
number_of_trajectory = len(concatDayWithID)
arrayForHeader = str(number_of_dimension) + '\n' + str(number_of_trajectory) + '\n'
print(concatDayWithID)
#format: TrackID no_trajectory_point X1 Y1 X2 Y2... Xn Yn
arrayForXY = []
number_of_ID = len(concatDayWithID)
counter_for_points = 0
counter_for_ID = 0
# Probably need to create another outer loop for diff sets of newList, like newList[1], newList[2]

while counter_for_ID < number_of_ID:
    # reset
    counter_for_points = 0
    # Convert ID to proper string form
    tempID = concatDayWithID[counter_for_ID]
    number_of_trajectory_points = len(newList[counter_for_ID])
#     print(number_of_trajectory_points)
    stringForTempID = str(tempID)
    stringForTempID = stringForTempID[1:-1]
    # concatenation
    if len(arrayForXY) == 0:
        arrayForXY = stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
    else:
        arrayForXY += stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
    while counter_for_points < number_of_trajectory_points:  
        # Select X and Y
        tempHelper = newList[counter_for_ID]
        tempArray = tempHelper.iloc[counter_for_points]
        helperX = tempArray[0]
        helperY = tempArray[1]
        arrayForXY += str(helperX) + ' ' + str(helperY) + ' '
        counter_for_points += 1
        if counter_for_points == number_of_trajectory_points:
            arrayForXY += '\n'
    counter_for_ID += 1

finalOutput = arrayForHeader + arrayForXY    

# print(newList[0])
# print(tempArray)
# print(arrayForHeader)
# print(arrayForXY)
# print(len(arrayForXY))
# print((concatDayWithID))
# print(finalOutput)

output = open('LOST2.tra', 'w')
output.write(finalOutput)
output.close()
# print(newList)



[[  157]
 [  734]
 [ 1040]
 [ 1280]
 [ 1294]
 [ 1305]
 [   68]
 [   97]
 [  132]
 [  370]
 [  527]
 [  552]
 [  714]
 [  749]
 [  774]
 [  781]
 [  817]
 [ 1218]
 [ 2071]
 [ 2218]
 [ 2234]
 [ 2235]
 [ 2282]
 [ 2715]
 [ 2733]
 [ 2813]
 [ 2818]
 [ 2881]
 [ 2948]
 [ 2959]
 [ 3001]
 [ 3312]
 [ 3495]
 [ 3581]
 [ 3587]
 [ 3593]
 [ 3634]
 [ 3678]
 [ 3735]
 [ 4156]
 [ 4212]
 [ 4359]
 [ 4409]
 [ 4412]
 [ 4617]
 [ 4686]
 [ 4767]
 [ 4951]
 [ 4956]
 [ 4959]
 [ 5016]
 [ 5041]
 [ 5247]
 [ 5622]
 [ 5628]
 [ 5639]
 [ 5678]
 [ 6013]
 [ 6149]
 [ 6161]
 [ 6194]
 [ 6293]
 [ 6364]
 [ 6546]
 [ 6641]
 [ 6678]
 [ 6765]
 [ 6775]
 [ 6785]
 [ 7107]
 [ 7194]
 [ 7257]
 [ 7961]
 [   30]
 [  261]
 [  325]
 [  456]
 [  480]
 [  540]
 [  785]
 [  905]
 [  939]
 [  958]
 [  998]
 [ 1238]
 [ 1386]
 [ 1447]
 [ 1463]
 [ 1499]
 [ 1608]
 [ 1783]
 [ 2127]
 [ 2428]
 [ 2559]
 [ 2760]
 [ 2972]
 [ 3003]
 [ 3021]
 [ 3372]
 [ 4077]
 [ 4295]
 [ 4297]
 [ 4366]
 [ 4479]
 [ 4494]
 [ 4524]
 [ 4531]
 [ 4582]
 [ 4601]
 [ 4666]
 [ 4941]
 

In [34]:
# Get the output from TraClus, revamp and prepare data to visualize
# to achieve this, i need to read from text file and reconstruct it back to a dataframe.
fileOpener = open('LOSTResult2.txt', 'r')
lines_ori = fileOpener.readlines()
# print(type(lines))

#Choose the rows with X Y coordinates
start = 2
end = len(lines_ori) 
step = 2
lines = lines_ori[start:end:step]
# print(lines)

#Choose the rows with ID and number of points
clusterID_array = []
num_points = []
start = 1
end = len(lines_ori)
step = 2
lines_ID = lines_ori[start:end:step]
length_lines_ID = len(lines_ID)
counter_for_linesID = 0
print(lines_ID[0])
while counter_for_linesID < length_lines_ID:
    temp_lines_ID = str(lines_ID[counter_for_linesID])
    temp_index = temp_lines_ID.find(" ")
    temp_index2 = temp_lines_ID.find("  ", 14, len(temp_lines_ID))
    clusterID_array += [temp_lines_ID[temp_index + 1]]
    num_points += [temp_lines_ID[temp_index2 + 2]]
    counter_for_linesID += 1
    
#Strip the string into appropriate form
element = str(lines)
elementText = element[0:-1]
elementText = str.replace(elementText, '\\' , '')
elementText = str.replace(elementText, "n", '')
elementText = str.replace(elementText, "   ", " ")
elementText = str.replace(elementText, ",", "\n")
elementText = str.replace(elementText, "\n", '')
elementText = str.replace(elementText, "'", "" )
elementText = str.replace(elementText, "  "," ")
elementText = str.replace(elementText, "[", "")
print(type(elementText))
print(elementText)

#put in 2d array form and convert to integer
numpyString = np.fromstring(elementText, dtype=float, sep= ' ' or '\n')
numpyInt = numpyString.astype(int)
number_of_rows = int( len(numpyInt)/2 )
numpyInt = numpyInt.reshape((number_of_rows,2))

#convert numpy array to dataframe
dataframeInt = pd.DataFrame(numpyInt)
dataframeInt.columns = ['X', 'Y']
dataframeInt = dataframeInt.convert_objects(convert_numeric=True)

#Assign ID to specific tracks
temp_array_ID = []
total_num_points = len(num_points)
counter_for_total_num = 0
while counter_for_total_num < total_num_points:
    tempCounter_for_num = 0
    while tempCounter_for_num < int(num_points[counter_for_total_num]):
        temp_array_ID += [clusterID_array[counter_for_total_num]]
        tempCounter_for_num += 1
    counter_for_total_num += 1

#finalized output
dataframeInt['TrackID'] = temp_array_ID
cols = dataframeInt.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataframeInt = dataframeInt[cols]
    
# print(numpyInt)
# print(dataframeInt.dtypes)
print(dataframeInt)
print(type(numpyInt))

# dummyTest2 = list(numpyInt)
# dummyTest2 = dataframeInt
# print(dummyTest2)

clusterID: 0  Points Number:  7

<class 'str'>
225.0006140310266 260.5880759640196 187.36905762732962 265.3757459320333 149.9437037130368 275.8951588158991 116.63986159887679 291.3293103604035 83.84848627324887 306.0293884269257 51.745446126150085 321.8013199359268 18.402172939240117 337.16203856717135 483.7309869111379 261.57887407486277 430.84240564293003 264.3505242751024 369.38679110375364 259.96822701809225 309.52534397941366 255.99253000571494 250.72340123969 253.1242332241077 194.78742892670817 254.37335990401954 
   TrackID    X    Y
0        0  225  260
1        0  187  265
2        0  149  275
3        0  116  291
4        0   83  306
5        0   51  321
6        0   18  337
7        1  483  261
8        1  430  264
9        1  369  259
10       1  309  255
11       1  250  253
12       1  194  254
<class 'numpy.ndarray'>


C:\Users\Ice\Anaconda3\lib\site-packages\ipykernel\__main__.py:55: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [35]:
# Construct lines for visualization in plot
# day1 = day1[['TrackID','X','Y']]
# dfPlotDay1 = day1[['X', 'Y']]
#assignment
# dataframeInt = concatDay

concatenator = dataframeInt[['TrackID','X','Y']]
concatenator2 = dataframeInt[['X', 'Y']]

row_iterator = concatenator.iterrows()
row_id, this_row = next(row_iterator)
#slicing dataframe to respective arrays for LineCollection function
initialID = this_row['TrackID']
slicingArray = []
for i, rows in row_iterator:
    if rows['TrackID'] != initialID:
        slicingArray.append(i)
        initialID = rows['TrackID']

testChamp = np.split( concatenator2, slicingArray )
testChampSize = len(testChamp)

###commented because not workable####
# indices = [0, 1]
# abc = np.take(testChamp[1], indices, axis = 1)
# abc = abc.as_matrix()
#####################################
dummyTest2 = list(testChamp)

print(testChamp)

[     X    Y
0  225  260
1  187  265
2  149  275
3  116  291
4   83  306
5   51  321
6   18  337,       X    Y
7   483  261
8   430  264
9   369  259
10  309  255
11  250  253
12  194  254]


In [36]:
#Clustering lines(Kmeans)
# indices = [1, 2]
indices = [0, 1]
counterForLineConstruction = 0

#line preparation
linesegment = mc.LineCollection(dummyTest2, linewidths = 2, linestyles='solid', colors='black')
# linesegment = mc.LineCollection(dataframeInt, linewidths = 2, linestyles='solid', colors='black')

#canvas setup
x = np.arange(641)
ys = x[0:480, np.newaxis]

# set plot limits
ax = plt.axes()
ax.set_xlim(x.min(), x.max())
ax.set_ylim(ys.min(), ys.max())
ax.add_collection(linesegment)
plt.gca().invert_yaxis()
im = plt.imread('overlayingImage.png')
implot = plt.imshow(im)
plt.show()

In [2]:
#Remake data for clustering purposes
#Clustering tracks for day1
day1 = day1[['X','Y']]
day2 = day2[['X','Y']]
day3 = day3[['X','Y']]
day4 = day4[['X','Y']]
day5 = day5[['X','Y']]
day6 = day6[['X','Y']]
day7 = day7[['X','Y']]

# concatDay = [day1, day2, day3, day4, day5, day6, day7]
concatDayTrain = [day1, day2, day3, day4, day5]
concatDayTrain = pd.concat(concatDayTrain)
concatDayTrain

concatDayTest = [day6, day7]
concatDayTest = pd.concat(concatDayTest)
concatDayTest

kmeans = KMeans(n_clusters=8, random_state=0).fit(concatDayTrain)
kmeansTrans = kmeans.transform(concatDayTrain)
centroids = kmeans.cluster_centers_
distance = kmeans.inertia_
labels = kmeans.labels_    #indication for each datapoint that determines the cluster it belongs to

tempDF = concatDayTrain
tempDF['cluster_label'] = Series(labels, index = tempDF.index)

#color code assignment
tempColor = np.where(tempDF['cluster_label'] == 0, 'black', 
                     np.where( tempDF['cluster_label'] == 1, 'yellow', 
                              np.where(tempDF['cluster_label'] == 2, 'magenta', 
                                       np.where(tempDF['cluster_label'] == 3, 'cyan', 
                                                np.where(tempDF['cluster_label'] == 4, 'red', 
                                                         np.where(tempDF['cluster_label'] == 5, 'green', 
                                                                 np.where(tempDF['cluster_label'] == 6, 'blue', 'orange'))))) ))


centroidsDF = pd.DataFrame(data=centroids[0:,0:], columns=['X','Y'])
# plt.scatter(centroidsDF['X'], centroidsDF['Y'], c='black', linewidths=50, marker='x')
# plt.scatter(concatDayTrain['X'], concatDayTrain['Y'], c=tempColor, s= 50) #red color dots as training set


print(kmeansTrans.size)
print(concatDayTrain.size)

# prediction
prediction = kmeans.predict(concatDayTest)
tempDF = concatDayTest
tempDF['cluster_label'] = Series(prediction, index = tempDF.index)

#color code assignment for prediction
tempColor = np.where(tempDF['cluster_label'] == 0, 'black', 
                     np.where( tempDF['cluster_label'] == 1, 'yellow', 
                              np.where(tempDF['cluster_label'] == 2, 'magenta', 
                                       np.where(tempDF['cluster_label'] == 3, 'cyan', 
                                                np.where(tempDF['cluster_label'] == 4, 'red', 
                                                         np.where(tempDF['cluster_label'] == 5, 'green', 
                                                                 np.where(tempDF['cluster_label'] == 6, 'blue', 'orange'))))) ))


plt.scatter(centroidsDF['X'], centroidsDF['Y'], c='black', linewidths=50, marker='x')
plt.scatter(tempDF['X'], tempDF['Y'], c=tempColor, s=50 )

# plt.gca().invert_yaxis()
plt.show()

im = plt.imread('overlayingImage.png')
implot = plt.imshow(im)

80928
30348
